In [9]:
import numpy as np
import pandas as pd

In [10]:
df = pd.read_csv('/home/k-k/MLP-Assignments/week3/Week3_GA_dataset.csv')
df.replace('?', np.nan, inplace=True)
d = df.drop('Target', axis=1)
d.head()

,V1,V2,V3,V4,V5
0,2.0,50.0,12500.0,98.0,NEGATIVE
1,0.0,13.0,3250.0,28.0,NEGATIVE
2,NaN,NaN,4000.0,35.0,NEGATIVE
3,NaN,20.0,5000.0,45.0,NEGATIVE
4,1.0,24.0,6000.0,77.0,NEGATIVE


In [11]:
df.isna().sum().sum()

np.int64(10)

In [12]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

processeor = ColumnTransformer([
    ('V1', pipeline, ['V1']),
    ('V2', pipeline, ['V2']),
    ('V3', StandardScaler(), ['V3']),
    ('V4', StandardScaler(), ['V4']),
    ('V5', OrdinalEncoder(), ['V5'])
], remainder='passthrough',verbose_feature_names_out=False)
processeor.set_output(transform='pandas')

final_pipeline = Pipeline([
    ('preprocessing', processeor),
    ('variance_filter', VarianceThreshold(threshold=0.1))
])

final = final_pipeline.fit_transform(d)
final.shape

(748, 4)

In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

processeor = ColumnTransformer([
    ('V1', pipeline, ['V1']),
    ('V2', pipeline, ['V2']),
    ('V3', StandardScaler(), ['V3']),
    ('V4', StandardScaler(), ['V4']),
    ('V5', OrdinalEncoder(), ['V5'])
], remainder='passthrough',verbose_feature_names_out=False)
processeor.set_output(transform='pandas')

model = LogisticRegression()
rfe = RFE(estimator=model, n_features_to_select=2)
final_pipeline = Pipeline([
    ('preprocessing', processeor),
    ('rfe',rfe)
])

X = df.drop('Target', axis=1)
y = OrdinalEncoder().fit_transform(df[['Target']]).ravel()

final_pipeline.fit(X,y)

selected_feature = final_pipeline.named_steps['rfe'].get_feature_names_out()
selected_feature


array(['V1', 'V3'], dtype=object)

In [14]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

processeor = ColumnTransformer([
    ('V1', pipeline, ['V1']),
    ('V2', pipeline, ['V2']),
    ('V3', StandardScaler(), ['V3']),
    ('V4', StandardScaler(), ['V4']),
    ('V5', OrdinalEncoder(), ['V5'])
], remainder='passthrough',verbose_feature_names_out=False)
processeor.set_output(transform='pandas')

model = LogisticRegression()
sfs = SequentialFeatureSelector(model, n_features_to_select=2, direction='forward', scoring='accuracy', cv=5)
final_pipeline = Pipeline([
    ('preprocessing', processeor),
    ('sfs',sfs)
])

X = df.drop('Target', axis=1)
y = OrdinalEncoder().fit_transform(df[['Target']]).ravel()

final_pipeline.fit(X,y)

selected_feature = final_pipeline.named_steps['sfs'].get_feature_names_out()
selected_feature

array(['V2', 'V4'], dtype=object)

In [15]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

processeor = ColumnTransformer([
    ('V1', pipeline, ['V1']),
    ('V2', pipeline, ['V2']),
    ('V3', StandardScaler(), ['V3']),
    ('V4', StandardScaler(), ['V4']),
    ('V5', OrdinalEncoder(), ['V5'])
], remainder='passthrough',verbose_feature_names_out=False)
processeor.set_output(transform='pandas')

model = LogisticRegression()
sfs = SequentialFeatureSelector(model, n_features_to_select=2, direction='backward', scoring='accuracy', cv=5)
final_pipeline = Pipeline([
    ('preprocessing', processeor),
    ('sfs',sfs)
])

X = df.drop('Target', axis=1)
y = OrdinalEncoder().fit_transform(df[['Target']]).ravel()

final_pipeline.fit(X,y)

selected_feature = final_pipeline.named_steps['sfs'].get_feature_names_out()
selected_feature

array(['V3', 'V4'], dtype=object)

In [16]:
df = df.drop(['PlayerID'], axis=1)
X = df.drop(['EngagementLevel'], axis=1)
y = df['EngagementLevel']

KeyError: "['PlayerID'] not found in axis"

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


impute = ColumnTransformer([
    ('Age', SimpleImputer(strategy='mean'), ['Age']),
    ('Location', SimpleImputer(strategy='constant', fill_value='Other'), ['Location']),
    ('GameDifficulty', SimpleImputer(strategy='most_frequent'), ['GameDifficulty']),
    ('InGamePurchases', SimpleImputer(strategy='constant', fill_value=0), ['InGamePurchases'])
], remainder='passthrough',verbose_feature_names_out=False)
impute.set_output(transform='pandas')

fit = impute.fit(X_train)
print(fit)
X_train_imputed = impute.fit_transform(X_train)
print(X_train_imputed)

X_test_imputed = impute.transform(X_test)



ColumnTransformer(remainder='passthrough',
                  transformers=[('Age', SimpleImputer(), ['Age']),
                                ('Location',
                                 SimpleImputer(fill_value='Other',
                                               strategy='constant'),
                                 ['Location']),
                                ('GameDifficulty',
                                 SimpleImputer(strategy='most_frequent'),
                                 ['GameDifficulty']),
                                ('InGamePurchases',
                                 SimpleImputer(fill_value=0,
                                               strategy='constant'),
                                 ['InGamePurchases'])],
                  verbose_feature_names_out=False)
       Age Location GameDifficulty  InGamePurchases  Gender   GameGenre  \
9254  34.0     Asia         Medium              0.0  Female  Simulation   
1561  41.0      USA         Medium         

## Ordinal Encoder

In [ ]:
ordinal_features = ['GameDifficulty']
nominal_features = ['Gender', 'Location', 'GameGenre']
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer

ordinal_encoder = OrdinalEncoder(
    categories=[['Easy', 'Medium', 'Hard']]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('ord', ordinal_encoder, ordinal_features),
        ('nom', OneHotEncoder(drop='first'), nominal_features),
        ('num', StandardScaler(), numeric_features)
    ],
    remainder='passthrough'
)

In [ ]:
X_train_transformed = preprocessor.fit_transform(X_train_imputed)
X_test_transformed = preprocessor.transform(X_test_imputed)


first_five = X_test_transformed[:5].sum()
first_five


np.float64(6.83718040298121)